In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
import pyanalib.pandas_helpers as ph
from makedf.util import *

import kinematics
import gump_cuts as gc

In [ ]:
FILE = "/Users/gputnam/Work/osc/cafpyana/analysis_village/gump/gumpdev.df"

In [ ]:
def load_data(file, nfiles=1):
    """Load event, header, and mcnu data from HDF file."""

    for s in range(nfiles):
        print("df index:"+str(s))
        df_evt = pd.read_hdf(file, "evt_"+str(s))
        df_hdr = pd.read_hdf(file, "hdr_"+str(s))
        df_mcnu = pd.read_hdf(file, "mcnu_"+str(s))
        df_stub = pd.read_hdf(file, "stub_"+str(s))

        matchdf = df_evt.copy()
        matchdf.columns = pd.MultiIndex.from_tuples([(col, '') for col in matchdf.columns])
        df_evt = ph.multicol_merge(matchdf.reset_index(), df_mcnu.reset_index(),
                                    left_on=[("__ntuple", ""), ("entry", ""), ("tmatch_idx", "")],
                                    right_on=[("__ntuple", ""), ("entry", ""), ("rec.mc.nu..index", "")],
                                    how="left") ## -- save all sllices

        cols_to_drop = []
        for c in df_evt.columns:
            if 'GENIE' in c[0] or 'Flux' in c[0]:
                cols_to_drop.append(c)

        df_evt.drop(columns=cols_to_drop, inplace=True)
        del df_mcnu

        if s == 0:
            res_df_evt = df_evt
            res_df_hdr = df_hdr
            res_df_stub = df_stub
        else:
            res_df_evt = pd.concat([res_df_evt, df_evt])
            res_df_hdr = pd.concat([res_df_hdr, df_hdr])
            res_df_stub = pd.concat([res_df_stub, df_stub])

        del df_evt
        del df_hdr
        del df_stub

    return res_df_evt, res_df_hdr, res_df_stub

In [ ]:
df, hdr, stub = load_data(FILE)

In [ ]:
df.columns = [c[0] for c in df.columns]

In [ ]:
def scale_pot(df, df_hdr, desired_pot):
    """Scale DataFrame by desired POT."""
    pot = sum(df_hdr.pot.tolist())
    print(f"POT: {pot}\nScaling to: {desired_pot}")
    scale = desired_pot / pot
    df['glob_scale'] = scale
    return pot, scale

In [ ]:
scale_pot(df, hdr, 6e20)

In [ ]:
glob_scale = df['glob_scale'].to_numpy()[0]
glob_scale

In [ ]:
mode_list = [0, 10, 1, 2, 3]
mode_labels = ['QE', 'MEC', 'RES', 'SIS/DIS', 'COH', "other"]

def breakdown_mode(var, df):
    """Break down variable by interaction mode."""
    ret = [var[df.genie_mode == i] for i in mode_list]
    ret.append(var[sum([df.genie_mode == i for i in mode_list]) == 0])
    return ret

In [ ]:
FONTSIZE = 14
HAWKS_COLORS = ["#315031", "#d54c28", "#1e3f54", "#c89648", "#43140b", "#95af8b"]

def add_style(ax, xlabel, title="", det="ICARUS"):
    ax.tick_params(axis='both', which='both', direction='in', length=6, width=1.5, labelsize=FONTSIZE, top=True, right=True)
    for spine in ax.spines.values():
        spine.set_linewidth(1.5)
    ax.set_xlabel(xlabel, fontsize=FONTSIZE, fontweight='bold')
    ax.set_ylabel('Events', fontsize=FONTSIZE, fontweight='bold')
    ax.set_title(f"$\\bf{{{det}}}$  {title}", fontsize=FONTSIZE+2)
    ax.legend(fontsize=FONTSIZE)

In [ ]:
FV = gc.slcfv_cut(df, "ICARUS") & gc.mufv_cut(df, "ICARUS") & gc.pfv_cut(df, "ICARUS") 

In [ ]:
cut = FV

In [ ]:
var = "nu_score"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0,1,21), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "$\\nu$ Score", "Contained")

In [ ]:
var = "crlongtrkdiry"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(-1,1,21), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "CRLongTrkDirY", "Fiducial")

In [ ]:
cut = FV & gc.cosmic_cut(df)

In [ ]:
var = "mu_len"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 500,21), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "Muon Length [cm]", title="Cosmic Rej.")

In [ ]:
var = "other_shw_length"

pvar = breakdown_mode(df.loc[cut, var].fillna(-1), df[cut])
n, bins, _ = plt.hist(pvar, bins=np.array([-10] + list(np.linspace(0, 200, 21))), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])
plt.yscale("log")
add_style(plt.gca(), "Maximum Shower Length [cm]", title="Cosmic Rej.")

In [ ]:
var = "other_trk_length"

pvar = breakdown_mode(df.loc[cut, var].fillna(-1), df[cut])
n, bins, _ = plt.hist(pvar, bins=np.array([-10] + list(np.linspace(0, 200, 21))), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])
plt.yscale("log")
add_style(plt.gca(), "Maximum Track Length [cm]", title="Cosmic Rej.")

In [ ]:
def twoprong_cut(df):
    return np.isnan(df.other_shw_length) & ~(df.other_trk_length > 10)

In [ ]:
cut = FV & gc.cosmic_cut(df) & twoprong_cut(df)

In [ ]:
var = "mu_len"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 500,21), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "Muon Length [cm]", title="Two Prong")

In [ ]:
var = "mu_chi2_of_mu_cand"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 60,21), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "Muon Candidate $\\chi^2_\\mu$", title="Two Prong")

In [ ]:
var = "prot_chi2_of_mu_cand"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 400,21), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "Muon Candidate $\\chi^2_p$", title="Two Prong")

In [ ]:
var = "mu_chi2_of_prot_cand"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 60,21), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "Proton Candidate $\\chi^2_\\mu$", title="Two Prong")

In [ ]:
var = "prot_chi2_of_prot_cand"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 400,21), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])


add_style(plt.gca(), "Proton Candidate $\\chi^2_p$", title="Two Prong")

In [ ]:
cut = FV & gc.cosmic_cut(df) & twoprong_cut(df) & gc.pid_cut_df(df)

In [ ]:
var = "mu_len"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 600, 11), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "Muon Length [cm]", title="PID")

In [ ]:
var = "del_p"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 1, 11), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "$\\delta p$ [GeV]", title="PID")

In [ ]:
var = "nu_E_calo"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 2, 16), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "$E_\\nu^\\mathrm{calo}$ [GeV]", title="PID")

In [ ]:
cut = FV & gc.cosmic_cut(df) & twoprong_cut(df) & gc.pid_cut_df(df) & (df.del_p < 0.25)

In [ ]:
var = "nu_E_calo"

pvar = breakdown_mode(df.loc[cut, var], df[cut])
n, bins, _ = plt.hist(pvar, bins=np.linspace(0, 2, 16), stacked=True, label=mode_labels,
                    color=HAWKS_COLORS, weights=[glob_scale*np.ones_like(p) for p in pvar])

add_style(plt.gca(), "$E_\\nu^\\mathrm{calo}$ [GeV]", title="PID")